In [1]:
# Based on
# https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/6.2-understanding-recurrent-neural-networks.ipynb

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

1.8.0


In [5]:
# https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification
max_features = 10000  # number of words to consider as features
maxlen = 500  # cut texts after this number of words (among top max_features most common words)

# each review is encoded as a sequence of word indexes
# indexed by overall frequency in the dataset
# output is 0 (negative) or 1 (positive) 
imdb = tf.keras.datasets.imdb.load_data(num_words=max_features)
(raw_input_train, y_train), (raw_input_test, y_test) = imdb

In [6]:
# tf.keras.datasets.imdb.load_data?

In [7]:
y_train.min()

0

In [8]:
y_train.max()

1

In [9]:
# 25000 texts
len(raw_input_train)

25000

In [10]:
# first text has 218 words
len(raw_input_train[0])

218

In [11]:
raw_input_train[0]

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 2,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 2,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 2,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,
 5535,
 18,

In [12]:
# tf.keras.preprocessing.sequence.pad_sequences?

In [13]:
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences

input_train = tf.keras.preprocessing.sequence.pad_sequences(raw_input_train, maxlen=maxlen)
input_test = tf.keras.preprocessing.sequence.pad_sequences(raw_input_test, maxlen=maxlen)

In [14]:
input_train.shape, input_test.shape, y_train.shape, y_test.shape

((25000, 500), (25000, 500), (25000,), (25000,))

In [15]:
# left padded with zeros
# As a convention, "0" does not stand for a specific word, but instead is used to encode any unknown word.
input_train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [23]:
# tf.keras.layers.SimpleRNN?
tf.keras.layers.Embedding?

In [28]:
model = tf.keras.Sequential()
# Parameters: max_features (10000) * 8 = 80000 
model.add(tf.keras.layers.Embedding(input_dim=max_features, output_dim=8, input_length=maxlen))
# model.add(tf.keras.layers.Embedding(max_features, 32, input_length=maxlen))
# model.add(tf.keras.layers.SimpleRNN(32, return_sequences=True))
# model.add(tf.keras.layers.SimpleRNN(32, return_sequences=True))
# model.add(tf.keras.layers.SimpleRNN(32))

# Input format: maxlen (500) * dimension of embedding (8)
# Output: 4000
model.add(tf.keras.layers.Flatten())

# binary classifier
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 500, 8)            80000     
_________________________________________________________________
flatten_6 (Flatten)          (None, 4000)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 4001      
Total params: 84,001
Trainable params: 84,001
Non-trainable params: 0
_________________________________________________________________


In [29]:
batch_size = 128

%time history = model.fit(input_train, y_train, epochs=10, batch_size=batch_size, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 2s 95us/step - loss: 0.7167 - acc: 0.5030 - val_loss: 0.7019 - val_acc: 0.5174
Epoch 2/10
20000/20000 [==============================] - 1s 67us/step - loss: 0.6907 - acc: 0.5440 - val_loss: 0.6909 - val_acc: 0.5414
Epoch 3/10
20000/20000 [==============================] - 1s 66us/step - loss: 0.6600 - acc: 0.6037 - val_loss: 0.6546 - val_acc: 0.6174
Epoch 4/10
20000/20000 [==============================] - 1s 68us/step - loss: 0.5887 - acc: 0.7130 - val_loss: 0.5916 - val_acc: 0.6744
Epoch 5/10
20000/20000 [==============================] - 1s 74us/step - loss: 0.5067 - acc: 0.7734 - val_loss: 0.4957 - val_acc: 0.7800
Epoch 6/10
20000/20000 [==============================] - 1s 67us/step - loss: 0.4164 - acc: 0.8380 - val_loss: 0.4313 - val_acc: 0.8138
Epoch 7/10
20000/20000 [==============================] - 1s 71us/step - loss: 0.3484 - acc: 0.8739 - val_loss: 0.3868 - val_acc:

In [27]:
import gc

del model
gc.collect()

tf.keras.backend.clear_session()

In [25]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(max_features, 32))
model.add(tf.keras.layers.SimpleRNN(32, return_sequences=True))
model.add(tf.keras.layers.SimpleRNN(32, return_sequences=True))
model.add(tf.keras.layers.SimpleRNN(32))

# binary classifier
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, None, 32)          2080      
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, None, 32)          2080      
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 326,273
Trainable params: 326,273
Non-trainable params: 0
_________________________________________________________________


In [26]:
batch_size = 128

%time history = model.fit(input_train, y_train, epochs=10, batch_size=batch_size, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 79s 4ms/step - loss: 0.6932 - acc: 0.5245 - val_loss: 0.6865 - val_acc: 0.5404
Epoch 2/10
20000/20000 [==============================] - 75s 4ms/step - loss: 0.6682 - acc: 0.5809 - val_loss: 0.6670 - val_acc: 0.5856
Epoch 3/10
20000/20000 [==============================] - 74s 4ms/step - loss: 0.5908 - acc: 0.6760 - val_loss: 0.6008 - val_acc: 0.6686
Epoch 4/10
20000/20000 [==============================] - 75s 4ms/step - loss: 0.4917 - acc: 0.7636 - val_loss: 0.5727 - val_acc: 0.7176
Epoch 5/10
20000/20000 [==============================] - 75s 4ms/step - loss: 0.3940 - acc: 0.8260 - val_loss: 0.4868 - val_acc: 0.7842
Epoch 6/10
20000/20000 [==============================] - 74s 4ms/step - loss: 0.3346 - acc: 0.8609 - val_loss: 0.5023 - val_acc: 0.7772
Epoch 7/10
20000/20000 [==============================] - 74s 4ms/step - loss: 0.3092 - acc: 0.8731 - val_loss: 0.5193 - val_acc:

## LSTMs

In [37]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(max_features, 32))
model.add(tf.keras.layers.LSTM(32))

# binary classifier
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 328,353
Trainable params: 328,353
Non-trainable params: 0
_________________________________________________________________


In [29]:
batch_size = 128

%time history = model.fit(input_train, y_train, epochs=10, batch_size=batch_size, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 136s 7ms/step - loss: 0.6782 - acc: 0.5729 - val_loss: 0.6345 - val_acc: 0.6514
Epoch 2/10
20000/20000 [==============================] - 135s 7ms/step - loss: 0.5924 - acc: 0.6887 - val_loss: 0.5230 - val_acc: 0.7508
Epoch 3/10
20000/20000 [==============================] - 138s 7ms/step - loss: 0.5083 - acc: 0.7577 - val_loss: 0.5250 - val_acc: 0.7372
Epoch 4/10
20000/20000 [==============================] - 138s 7ms/step - loss: 0.4384 - acc: 0.8012 - val_loss: 0.4658 - val_acc: 0.7786
Epoch 5/10
20000/20000 [==============================] - 163s 8ms/step - loss: 0.4032 - acc: 0.8216 - val_loss: 0.3855 - val_acc: 0.8252
Epoch 6/10
20000/20000 [==============================] - 145s 7ms/step - loss: 0.3626 - acc: 0.8457 - val_loss: 0.3903 - val_acc: 0.8314
Epoch 7/10
20000/20000 [==============================] - 138s 7ms/step - loss: 0.3298 - acc: 0.8616 - val_loss: 0.3690 - v

In [36]:
train_loss, train_accuracy = model.evaluate(input_train, y_train, batch_size=batch_size)
train_accuracy

25000/25000 [==============================] - 50s 2ms/step


0.80948

In [31]:
test_loss, test_accuracy = model.evaluate(input_test, y_test, batch_size=batch_size)
test_accuracy

25000/25000 [==============================] - 50s 2ms/step


0.7776799999809265

In [34]:
# precition
model.predict(input_test[0:5])

array([[0.1902623 ],
       [0.7821825 ],
       [0.24731928],
       [0.24368538],
       [0.96904045]], dtype=float32)

In [35]:
# ground truth
y_test[0:5]

array([0, 1, 1, 0, 1], dtype=int64)

## GRU

In [1]:
drop_out = 0.15

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(max_features, 32))
model.add(tf.keras.layers.Dropout(drop_out))
model.add(tf.keras.layers.GRU(32))
model.add(tf.keras.layers.Dropout(drop_out))

# binary classifier
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

NameError: name 'tf' is not defined

In [42]:
batch_size = 1000

%time history = model.fit(input_train, y_train, epochs=25, batch_size=batch_size, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/25
20000/20000 [==============================] - 15s 729us/step - loss: 0.7017 - acc: 0.5153 - val_loss: 0.6872 - val_acc: 0.5490
Epoch 2/25
20000/20000 [==============================] - 14s 682us/step - loss: 0.6843 - acc: 0.5529 - val_loss: 0.6840 - val_acc: 0.5476
Epoch 3/25
20000/20000 [==============================] - 14s 684us/step - loss: 0.6762 - acc: 0.5859 - val_loss: 0.6768 - val_acc: 0.5694
Epoch 4/25
20000/20000 [==============================] - 14s 696us/step - loss: 0.6653 - acc: 0.6095 - val_loss: 0.6673 - val_acc: 0.5830
Epoch 5/25
20000/20000 [==============================] - 13s 668us/step - loss: 0.6451 - acc: 0.6385 - val_loss: 0.6485 - val_acc: 0.6100
Epoch 6/25
20000/20000 [==============================] - 13s 661us/step - loss: 0.5966 - acc: 0.6859 - val_loss: 0.5830 - val_acc: 0.6878
Epoch 7/25
20000/20000 [==============================] - 13s 659us/step - loss: 0.5074 - acc: 0.7542 - val_loss: 0.5